In [20]:
class LSTM_output_layer(nn.Module):
    def __init__(self, hidden_size, output_size, max_sentence_length):
        super(LSTM_output_layer, self).__init__()

        self.output_size = output_size
        self.hidden_size = hidden_size
        
        self.activation = nn.ReLU()
        
        self.final_activation = nn.Sigmoid()
        
        self.i2o = nn.Sequential(
            nn.Linear(max_sentence_length*hidden_size,
                      max_sentence_length*hidden_size//2),
            self.activation,
            nn.Linear(max_sentence_length*hidden_size//2,
                      output_size*4),
            self.activation,
            nn.Linear(output_size*4,output_size),
            self.final_activation)

    def forward(self, input):
        input = input.reshape(input.shape[0],-1)
        output = self.i2o(input)
        
        return output

class LSTM(nn.Module):
    def __init__(self, embeddingm batch_size=30, input_size=25, hidden_size=25, output_size=1, num_layers=1, max_sentence_length = 20):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size        
        
        self.LSTM_intermediary = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True)
        
        self.embedding = embedding
        
        self.LSTM_final = LSTM_output_layer(hidden_size, output_size, max_sentence_length)
        
        
    def forward(self, inputs):
        
        inputs_embedded = self.embedding(inputs)
        
        sentence_lengths = (inputs_embedded.sum(dim=2,keepdim=False)!=0).sum(dim=1)
        
        output, (hidden, cell_state) = self.LSTM_intermediary(inputs_embedded)
        print("Output shape: ",output.shape)
        
        output = self.LSTM_final(output)
        
        return output
    
    def loss_fn(self):
        ''' Returns the loss function best associated with the model'''
        return nn.BCELoss()

In [ ]:
import torch.nn as nn

class RNN_building_block(nn.Module):
    def __init__(self, input_size, output_size):
        super(RNN_building_block, self).__init__()

        self.i2h = nn.Linear(input_size, output_size)
        
    def forward(self, input):

        output = self.i2h(input)
               
        return output
    
    
class RNN_hidden_layer(nn.Module):
    def __init__(self, input_size, hidden_size, intermediary_layer_sizes=[]):
        super(RNN_hidden_layer, self).__init__()

        self.hidden_size = hidden_size
        
        self.activation = nn.ReLU()
        
        self.component_layers=[]
        if intermediary_layer_sizes!=[]:
            
            self.component_layers.append(
                RNN_building_block(input_size = input_size + hidden_size,
                                   output_size = intermediary_layer_sizes[0]))
            self.component_layers.append(self.activation)
                
                
            if len(intermediary_layer_sizes)>1:
                for layer in range(1,len(intermediary_layer_sizes)):
                    self.component_layers.append(
                        RNN_building_block(
                                input_size = intermediary_layer_sizes[layer-1],
                                output_size = intermediary_layer_sizes[layer]))
                    self.component_layers.append(self.activation)
                    
            self.component_layers.append(
                RNN_building_block(
                        input_size = intermediary_layer_sizes[-1],
                        output_size = hidden_size))
                
        else:
            self.component_layers.append(
                RNN_building_block(input_size = input_size + hidden_size,
                                   output_size = hidden_size))
            
        self.i2h = nn.Sequential(*self.component_layers)

        
    def forward(self, input, hidden):
        #print(input.shape)
        #print(hidden.shape)
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
               
        return hidden
    
class RNN_output_layer(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(RNN_output_layer, self).__init__()

        self.output_size = output_size
        self.hidden_size = hidden_size
        
        self.activation=nn.ReLU()
        
        self.i2o = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            self.activation,
            nn.Linear(hidden_size, hidden_size//2),
            self.activation,
            nn.Linear(hidden_size//2, output_size))
        self.final_activation = nn.Sigmoid()

    def forward(self, hidden): 
        output = self.i2o(hidden)
        output = self.final_activation(output)
        
        return output

class RNN(nn.Module):
    def __init__(self, embedding, batch_size=16, intermediary_dims=[25,25,25], input_size=25, hidden_size=25, output_size=1):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.embedding = embedding
        
        #Convolutional layers
        
        self.rnn_hidden = RNN_hidden_layer(input_size, hidden_size,intermediary_dims)
        self.rnn_final = RNN_output_layer(hidden_size, output_size)
        
        
    def forward(self, inputs):
        
        hidden = torch.zeros(inputs.shape[0],self.hidden_size)
        
        inputs_embedded = self.embedding(inputs)
        
        sentence_lengths = (inputs_embedded.sum(dim=2,keepdim=False)!=0).sum(dim=1)
        
        final_hidden = torch.zeros(inputs_embedded.shape[0],self.hidden_size)
        
        for word_no in range(inputs.shape[1]):
            hidden = self.rnn_hidden(inputs_embedded[:,word_no,:],hidden)
            
            is_final_word = (sentence_lengths==word_no+1).nonzero().flatten()

            final_hidden[is_final_word] = hidden[is_final_word] 

        output = self.rnn_final(final_hidden)
        
        return output
    
    def loss_fn(self):
        ''' Returns the loss function best associated with the model'''
        return nn.BCELoss()

In [ ]:
        emoji_dict =  {'♥️': ' love ',
                       '❤️' : ' love ',
                       '❤' : ' love ',
                       '😘' : ' kisses ',
                      '😭' : ' cry ',
                      '💪' : ' strong ',
                      '🌍' : ' earth ',
                      '💰' : ' money ',
                      '👍' : ' ok ',
                       '👌' : ' ok ',
                      '😡' : ' angry ',
                      '🍆' : ' dick ',
                      '🤣' : ' haha ',
                      '😂' : ' haha ',
                      '🖕' : ' fuck you ',
                      '💩': 'shit'}

In [ ]:
        elif cType == 'RNN':
            self.model = RNN(self.embedding, 1000, [200,200], dim_vect, 200, 1)
        elif cType == "LSTM":
            self.model = LSTM(self.embedding,  batch_size=16, input_size=dim_vect, hidden_size=dim_vect, output_size=1, num_layers=1)
        
